In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict
from deep_sort_realtime.deepsort_tracker import DeepSort

# 필요한 상수 및 설정
CONFIDENCE_THRESHOLD = 0.4
MIN_BOX_AREA = 5000  # 최소 박스 면적 설정

# 클래스 리스트 로드 함수
def load_class_list(file_path):
    with open(file_path, 'r') as f:
        return f.read().split('\n')

# 모델 초기화 함수
def initialize_yolo(model_path):
    model = YOLO(model_path)
    return model

# 이미지 파일 목록 가져오기 함수
def get_image_files(image_folder_path):
    file_names = os.listdir(image_folder_path)
    image_files = [f for f in file_names if f.endswith(('.jpg', '.png'))]
    image_files.sort()
    return image_files

# IOU 계산 함수
def calculate_iou(box1, box2):
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    inter_area = max(0, inter_x_max - inter_x_min) * max(0, inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)

    return inter_area / (box1_area + box2_area - inter_area) if (box1_area + box2_area - inter_area) > 0 else 0

# 포함 비율 계산 함수
def calculate_containment_ratio(outer_box, inner_box):
    x1_min, y1_min, x1_max, y1_max = outer_box
    x2_min, y2_min, x2_max, y2_max = inner_box

    inner_area = (x2_max - x2_min) * (y2_max - y2_min)
    outer_area = (x1_max - x1_min) * (y1_max - y1_min)

    if outer_area > 0:
        containment_ratio = inner_area / outer_area
        return containment_ratio >= 0.90
    return False

# 바운딩 박스 그룹화 함수
def group_boxes(boxes, labels, confidences, iou_min=0.02, iou_max=0.13):
    groups = []
    used = set()

    for i, box in enumerate(boxes):
        if i in used or labels[i] != 0:
            continue
        group = [(box, confidences[i], labels[i])]
        used.add(i)

        blinkers = []
        for j, other_box in enumerate(boxes):
            if j != i and j not in used and labels[j] == 1:
                iou = calculate_iou(box, other_box)
                if iou_min <= iou <= iou_max or calculate_containment_ratio(box, other_box):
                    blinkers.append((other_box, confidences[j], labels[j]))
                    used.add(j)

        blinkers = blinkers[:2]
        group.extend(blinkers)

        groups.append(group)

    return groups

# 라벨 업데이트 함수 수정
def update_labels(grouped_boxes):
    updated_boxes = []

    for idx, group in enumerate(grouped_boxes):
        car_label = f"car{idx + 1}"

        # 그룹 내 객체를 클래스별로 분류
        car_items = []
        blinker_items = []

        for item in group:
            box, conf, label = item
            if label == 0:  # 자동차 클래스
                car_items.append((box, conf, label))
            elif label == 1:  # 방향지시등 클래스
                blinker_items.append((box, conf, label))

        # 자동차 객체는 하나라고 가정
        if not car_items:
            continue  # 자동차가 없으면 스킵

        car_item = car_items[0]
        car_box = car_item[0]
        car_x_center = (car_box[0] + car_box[2]) / 2

        updated_group = []

        # 자동차 라벨 추가
        updated_group.append({
            'original_label': int(car_item[2]),
            'new_label': f"{car_label}-1",
            'box': car_item[0],
            'confidence': car_item[1]
        })

        # 방향지시등 라벨 할당
        for blinker_item in blinker_items:
            blinker_box = blinker_item[0]
            blinker_x_center = (blinker_box[0] + blinker_box[2]) / 2


            # 자동차가 카메라로부터 멀어지는 방향이라고 가정
            if blinker_x_center < car_x_center:
                blinker_label = f"{car_label}-L"
            else:
                blinker_label = f"{car_label}-R"

            updated_group.append({
                'original_label': int(blinker_item[2]),
                'new_label': blinker_label,
                'box': blinker_item[0],
                'confidence': blinker_item[1]
            })

        updated_boxes.append(updated_group)

    return updated_boxes

# 함수 A: 라벨 수가 3개인 그룹 처리
def A(group, img, idx):
    # 방향지시등 판단을 위한 변수 초기화
    left_blinker_on = False
    right_blinker_on = False

    # 그룹 내 라벨을 통해 방향지시등 판단
    for item in group:
        new_label = item['new_label']
        if '-L' in new_label:
            left_blinker_on = True
        elif '-R' in new_label:
            right_blinker_on = True

    # 방향지시등 상태 출력 메시지 생성
    output_message = f"프레임 {idx + 1}: "
    if left_blinker_on:
        output_message += "왼쪽 깜빡이가 켜졌습니다 - 속도를 줄이시오"
    elif right_blinker_on:
        output_message += "오른쪽 깜빡이가 켜졌습니다 - 속도를 줄이시오"
    elif left_blinker_on and right_blinker_on:
        output_message += "양쪽 깜빡이가 켜졌습니다 - 속도를 줄이시오"
    else:
        output_message += "방향지시등이 꺼져 있습니다 - 속도를 유지하시오"

    # 결과 출력
    print(output_message)

def B(group, frames, idx):
    # 필요한 상수 및 함수 정의
    angle_ranges = {
        'left': [
            ('좌회전', [(135, 145), (170, 195)]),
            ('우회전', [(0, 20)]),
            ('정차', [(0, 20), (170, 174), (330, 350)]),
            ('반대', [(160, 170), (174, 189), (194, 200)]),
            ('주행', [(0, 115), (115, 360)]),
        ],
        'right': [
            ('좌회전', [(0, 0)]),
            ('우회전', [(0, 5), (10, 15), (330, 360)]),
            ('정차', [(0, 100), (315, 330)]),
            ('주행', [(15, 315)]),
            ('반대', [(180, 180)])
        ]
    }

    def is_small_box(box, min_box_area=MIN_BOX_AREA):
        x1, y1, x2, y2 = map(int, box)
        box_area = (x2 - x1) * (y2 - y1)
        return box_area <= min_box_area

    def calculate_angle(vector):
        angle = np.degrees(np.arctan2(vector[1], vector[0]))
        if angle < 0:
            angle += 360
        return angle

    def classify_direction(angle, side):
        for status, ranges in angle_ranges[side]:
            for min_angle, max_angle in ranges:
                if min_angle <= angle <= max_angle:
                    return status
        return 'Unknown'

    def dense_optical_flow(img1, img2, box):
        x1, y1, x2, y2 = box
        height, width = img1.shape[:2]
        x1, y1 = max(0, int(x1)), max(0, int(y1))
        x2, y2 = min(width, int(x2)), min(height, int(y2))

        img1_cropped = img1[y1:y2, x1:x2]
        img2_cropped = img2[y1:y2, x1:x2]

        if img1_cropped.size == 0 or img2_cropped.size == 0:
            return np.array([0, 0]), 0, (x1, y1, x2, y2)

        img1_gray = cv2.cvtColor(img1_cropped, cv2.COLOR_BGR2GRAY)
        img2_gray = cv2.cvtColor(img2_cropped, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(
            img1_gray, img2_gray, None,
            0.5, 3, 15, 3, 5, 1.2, 0
        )
        flow_x = np.mean(flow[..., 0])
        flow_y = np.mean(flow[..., 1])
        flow_magnitude = np.sqrt(flow_x ** 2 + flow_y ** 2)
        return np.array([flow_x, flow_y]), flow_magnitude, (x1, y1, x2, y2)

    # Optical Flow와 박스 처리
    frame1 = frames[idx].copy()
    frame2 = frames[idx + 1].copy() if idx + 1 < len(frames) else frames[idx].copy()
    height, width = frame1.shape[:2]
    mid_x = width // 2

    frame_statuses = []

    for item in group:
        box = item['box']
        if is_small_box(box):
            continue

        flow_vector, flow_magnitude, (x1, y1, x2, y2) = dense_optical_flow(
            frame1, frame2, box)
        angle = calculate_angle(flow_vector)
        side = 'left' if (x1 + x2) // 2 < mid_x else 'right'

        status = classify_direction(angle, side)
        frame_statuses.append(status)

    # 출력 메시지 생성
    output_message = f"프레임 {idx + 1}: "
    messages = []
    for status in set(frame_statuses):  # 중복 제거
        if status in ['좌회전', '우회전', '정차']:
            messages.append(f"{status} - 속도를 줄이시오")
        elif status in ['반대', '주행']:
            messages.append(f"{status} - 속도를 유지하시오")
    if messages:
        output_message += ", ".join(messages)
    else:
        output_message += "감지된 차량이 없습니다."

    # 결과 출력
    print(output_message)

# 함수 C: 라벨 수가 1개인 그룹 처리
def C(group, frames, idx):
    # 필요한 상수 및 함수 정의는 함수 B와 동일합니다.
    angle_ranges = {
        'left': [
            ('좌회전', [(135, 145), (170, 195)]),
            ('우회전', [(0, 20)]),
            ('정차', [(0, 20), (170, 174), (330, 350)]),
            ('반대', [(160, 170), (174, 189), (194, 200)]),
            ('주행', [(0, 115), (115, 360)]),
        ],
        'right': [
            ('좌회전', [(0, 0)]),
            ('우회전', [(0, 5), (10, 15), (330, 360)]),
            ('정차', [(0, 100), (315, 330)]),
            ('주행', [(15, 315)]),
            ('반대', [(180, 180)])
        ]
    }

    def is_small_box(box, min_box_area=MIN_BOX_AREA):
        x1, y1, x2, y2 = map(int, box)
        box_area = (x2 - x1) * (y2 - y1)
        return box_area <= min_box_area

    def calculate_angle(vector):
        angle = np.degrees(np.arctan2(vector[1], vector[0]))
        if angle < 0:
            angle += 360
        return angle

    def classify_direction(angle, side):
        for status, ranges in angle_ranges[side]:
            for min_angle, max_angle in ranges:
                if min_angle <= angle <= max_angle:
                    return status
        return 'Unknown'

    def dense_optical_flow(img1, img2, box):
        x1, y1, x2, y2 = box
        height, width = img1.shape[:2]
        x1, y1 = max(0, int(x1)), max(0, int(y1))
        x2, y2 = min(width, int(x2)), min(height, int(y2))

        img1_cropped = img1[y1:y2, x1:x2]
        img2_cropped = img2[y1:y2, x1:x2]

        if img1_cropped.size == 0 or img2_cropped.size == 0:
            return np.array([0, 0]), 0, (x1, y1, x2, y2)

        img1_gray = cv2.cvtColor(img1_cropped, cv2.COLOR_BGR2GRAY)
        img2_gray = cv2.cvtColor(img2_cropped, cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(
            img1_gray, img2_gray, None,
            0.5, 3, 15, 3, 5, 1.2, 0
        )
        flow_x = np.mean(flow[..., 0])
        flow_y = np.mean(flow[..., 1])
        flow_magnitude = np.sqrt(flow_x ** 2 + flow_y ** 2)
        return np.array([flow_x, flow_y]), flow_magnitude, (x1, y1, x2, y2)

    # Optical Flow와 박스 처리
    frame1 = frames[idx].copy()
    frame2 = frames[idx + 1].copy() if idx + 1 < len(frames) else frames[idx].copy()
    height, width = frame1.shape[:2]
    mid_x = width // 2

    frame_statuses = []

    for item in group:
        box = item['box']
        if is_small_box(box):
            continue

        flow_vector, flow_magnitude, (x1, y1, x2, y2) = dense_optical_flow(
            frame1, frame2, box)
        angle = calculate_angle(flow_vector)
        side = 'left' if (x1 + x2) // 2 < mid_x else 'right'

        status = classify_direction(angle, side)
        frame_statuses.append(status)

    # 출력 메시지 생성
    output_message = f"프레임 {idx + 1}: "
    messages = []
    for status in set(frame_statuses):  # 중복 제거
        if status in ['좌회전', '우회전', '정차']:
            messages.append(f"{status} - 속도를 줄이시오")
        elif status in ['반대', '주행']:
            messages.append(f"{status} - 속도를 유지하시오")
    if messages:
        output_message += ", ".join(messages)
    else:
        output_message += "감지된 차량이 없습니다."

    # 결과 출력
    print(output_message)

# 메인 함수
def main():
    IMAGE_FOLDER_PATH = '/Users/kimyu/Desktop/trainn/test2/test2-10'
    MODEL_PATH = '/Users/kimyu/Desktop/융캡디/yun/best.pt'
    CLASS_LIST_PATH = '/Users/kimyu/Desktop/융캡디/yun/yolo_class.txt'

    class_list = load_class_list(CLASS_LIST_PATH)
    model = initialize_yolo(MODEL_PATH)
    image_files = get_image_files(IMAGE_FOLDER_PATH)
    frames = []

    processed_results = []

    for idx, image_file in enumerate(image_files):
        image_path = os.path.join(IMAGE_FOLDER_PATH, image_file)
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Could not load image {image_path}")
            continue

        frames.append(img)

        # 모델 예측
        result = model.predict(source=[img], save=False, verbose=False)[0]

        if result.orig_img is not None and result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            labels = result.boxes.cls.cpu().numpy().astype(int)
            confidences = result.boxes.conf.cpu().numpy()

            if len(boxes) == 0:
                print(f"No detections in image {idx}")
                continue

            # 바운딩 박스 그룹화 및 라벨 업데이트
            grouped_boxes = group_boxes(boxes, labels, confidences)
            updated_grouped_boxes = update_labels(grouped_boxes)

            # 그룹화된 결과 저장
            processed_results.append(updated_grouped_boxes)

            # 각 그룹에 대해 라벨 수에 따라 함수 호출
            for group in updated_grouped_boxes:
                num_labels = len(group)
                if num_labels == 1:
                    C(group, frames, idx)
                elif num_labels == 2:
                    B(group, frames, idx)
                elif num_labels == 3:
                    A(group, img, idx)
                else:
                    print(f"프레임 {idx + 1}: 감지된 그룹이 없습니다.")
        else:
            processed_results.append([])

if __name__ == "__main__":
    main()




0: 384x640 2 Cars, 4 Lights, 169.8ms
Speed: 1.5ms preprocess, 169.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 169.2ms
Speed: 1.4ms preprocess, 169.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 180.0ms
Speed: 1.3ms preprocess, 180.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 5 Lights, 172.9ms
Speed: 1.3ms preprocess, 172.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 5 Lights, 162.2ms
Speed: 1.4ms preprocess, 162.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 4 Lights, 161.3ms
Speed: 1.3ms preprocess, 161.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 5 Lights, 149.1ms
Speed: 1.6ms preprocess, 149.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 Cars, 5 Lights, 151.6ms
Speed: 1.5